# Filling out incomplete coloc groupings 
I found some groupings that were incomplete; that is a certain groupings were defined only by a number of pairwise relationships that is less than nC2 (n choose 2) relaionships for a grouping of n stain types. AG confirmed or amended these groupings and here I will stitch togeter the final set and begin asking some our more interesting coloc questions discussed in the previous notebook.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import sys
import statsmodels.api as sm 
from statsmodels.formula.api import ols 

# loading some functions we wrote before
sys.path.append("/Users/jonathanramos/Desktop/LRI/Image ROI Data Wrangling/")
from clean import *
from norm import *
from count import *

/var/folders/b2/3h2lpxx14kgb12pp_7pltxnc0000gn/T/ipykernel_9974/543169497.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Loading in sets

We want to merge these sets so that all cells have fully updated groups (sets of unique cell roi ids) that they are associated with. Then we can build new dummy cols and start digging into the data.

We also have the following updated unpaired doubles:

- KET-10-12_PFC_3.9_C: expected the following missing pairs: [('0-FFF-00061_Npas4', '0-005-00025_cFos')] YES
- KET-10-3_PFC_3.5_C: expected the following missing pairs: [('0-FFF-00064_Npas4', '0-FFF-00070_cFos')] YES
- KET-10-3_PFC_3.7_D: expected the following missing pairs: [('0-FFF-00048_Npas4', '0-005-00024_cFos’)] YES
- KET-8-2_PFC_4.0_A: expected the following missing pairs: [('0-FFF-00103_Npas4', '0-005-00015_cFos’)] YES
- PE-11-1_PFC_3.6_B: expected the following missing pairs: [('0-200-00007_Npas4', '0-005-00026_cFos')] NO
- PE-11-3_PFC_3.5_C: expected the following missing pairs: [('0-200-00000_Npas4', '0-FFF-00033_cFos')] NO
- PE-11-5_PFC_3.7_D: expected the following missing pairs: [('0-FFF-00022_Npas4', '0-FFF-00004_cFos’)] YES

## Full cleaned set from previous notebook

In [2]:
df_coloc = pd.read_csv('KET-VR5_COLOC_FULL_DB.csv')

# let's take a look
print(df_coloc.shape)
print(df_coloc.columns)
df_coloc

(22798, 19)
Index(['Unnamed: 0', 'roi_id', 'coloc_w/_PV', 'coloc_w/_cFos',
       'coloc_w/_Npas4', 'coloc_w/_WFA', 'CoM_x', 'CoM_y', 'background',
       'mean_intensity', 'filename', 'rat_n', 'treatment', 'stain_type',
       'image_name', 'dummy_PV', 'dummy_cFos', 'dummy_Npas4', 'dummy_WFA'],
      dtype='object')


,Unnamed: 0,roi_id,coloc_w/_PV,coloc_w/_cFos,coloc_w/_Npas4,coloc_w/_WFA,CoM_x,CoM_y,background,mean_intensity,filename,rat_n,treatment,stain_type,image_name,dummy_PV,dummy_cFos,dummy_Npas4,dummy_WFA
0,0,0-000-00000_PV,-,-,0-FFF-00045_Npas4,0-FFF-00004_WFA,297.86,425.97,238.8715,536.8331,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,PV,KET-10-12_PFC_3.7_A,False,False,True,True
1,1,0-000-00001_PV,-,0-FFF-00070_cFos,0-FFF-00012_Npas4,-,340.47,43.89,238.8715,314.9278,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,PV,KET-10-12_PFC_3.7_A,False,True,True,False
2,2,0-000-00002_PV,-,-,0-FFF-00044_Npas4,0-FFF-00005_WFA,154.85,476.15,238.8715,324.0556,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,PV,KET-10-12_PFC_3.7_A,False,False,True,True
3,3,0-000-00003_PV,-,-,0-FFF-00082_Npas4,0-FFF-00003_WFA,310.10,308.22,238.8715,346.0313,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,PV,KET-10-12_PFC_3.7_A,False,False,True,True
4,4,0-000-00004_PV,-,-,-,-,44.35,323.64,238.8715,429.6127,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,PV,KET-10-12_PFC_3.7_A,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22793,0,-,-,-,-,-,177.27,68.60,93.9887,88.2249,PE-13-11_PFC_3.3_C_5.tif,PE-13-11,VR5_SAL,WFA,PE-13-11_PFC_3.3_C,False,False,False,False
22794,8,-,-,-,-,-,164.01,117.08,111.9724,146.2031,PE-13-11_PFC_3.5_D_5.tif,PE-13-11,VR5_SAL,WFA,PE-13-11_PFC_3.5_D,False,False,False,False
22795,9,-,-,-,-,-,197.94,107.45,111.9724,123.3614,PE-13-11_PFC_3.5_D_5.tif,PE-13-11,VR5_SAL,WFA,PE-13-11_PFC_3.5_D,False,False,False,False
22796,10,-,-,-,-,-,229.32,96.22,111.9724,108.3670,PE-13-11_PFC_3.5_D_5.tif,PE-13-11,VR5_SAL,WFA,PE-13-11_PFC_3.5_D,False,False,False,False


## Amended groupings, confirmed by visual inspection of images

In [3]:
# reading in our set of amended groupings
paired_quads = pd.read_csv('PAIRED_unpaired_triples_quads/unpaired_quads.csv')
paired_triples = pd.read_csv('PAIRED_unpaired_triples_quads/unpaired_triples.csv')

# from the text above, building a df of amended doubles
doubles = [
    ('KET-10-12_PFC_3.9_C', ('0-FFF-00061_Npas4', '0-005-00025_cFos'), True), #YES
    ('KET-10-3_PFC_3.5_C', ('0-FFF-00064_Npas4', '0-FFF-00070_cFos'), True), #YES
    ('KET-10-3_PFC_3.7_D', ('0-FFF-00048_Npas4', '0-005-00024_cFos'), True), #YES
    ('KET-8-2_PFC_4.0_A', ('0-FFF-00103_Npas4', '0-005-00015_cFos'), True), #YES
    ('PE-11-1_PFC_3.6_B', ('0-200-00007_Npas4', '0-005-00026_cFos'), False), #NO
    ('PE-11-3_PFC_3.5_C', ('0-200-00000_Npas4', '0-FFF-00033_cFos'), False), #NO
    ('PE-11-5_PFC_3.7_D', ('0-FFF-00022_Npas4', '0-FFF-00004_cFos'), True) #YES
    ]

paired_doubles = pd.DataFrame(doubles, columns = ['image_name', 'grouping', 'correct_grouping'])

# standardizing "Correct grouping? (yes/no)" col name
paired_triples['correct_grouping'] = paired_triples['Correct grouping? (yes/no)']\
    .apply(lambda x: True if x.lower() == 'yes' else False)
paired_quads['correct_grouping'] = paired_quads['Correct grouping? (yes/no)']\
    .apply(lambda x: True if x.lower() == 'yes' else False)

# interpretting grouping string as python object
sort_order = {'PV': 0, 'cFos': 1, 'Npas4': 2, 'WFA': 3}
paired_quads['grouping'] = paired_quads.grouping.apply(
    lambda x: tuple(sorted(list(ast.literal_eval(x)),\
                            key=lambda y: sort_order[y.split('_')[-1]])))

paired_triples['grouping'] = paired_triples.grouping.apply(
    lambda x: tuple(sorted(list(ast.literal_eval(x)),\
                            key=lambda y: sort_order[y.split('_')[-1]])))

# converting amended grouping cols to tuples from strings
for col in [c for c in paired_triples.columns if 'amended' in c]:
    paired_triples[col] = paired_triples[col].apply(
        lambda x: tuple(sorted(
            x.replace("'", "").replace(' ', '').strip(',').split(','),\
                  key=lambda y: sort_order[y.split('_')[-1]]
            )) if type(x) == str else x)
    
for col in [c for c in paired_quads.columns if 'amended' in c]:
    paired_quads[col] = paired_quads[col].apply(
        lambda x: tuple(sorted(
            x.replace("'", "").replace(' ', '').strip(',').split(','),\
                  key=lambda y: sort_order[y.split('_')[-1]]
            )) if type(x) == str else x)
    
# building new roi_id col by exploding out grouping col
paired_doubles['roi_id'] = paired_doubles.grouping
paired_doubles = paired_doubles.explode('roi_id').reset_index()

paired_triples['roi_id'] = paired_triples.grouping
paired_triples = paired_triples.explode('roi_id').reset_index()

paired_quads['roi_id'] = paired_quads.grouping
paired_quads = paired_quads.explode('roi_id').reset_index()

# aggregating amended grouping columns into one 
def agg_amend_dbls(x):
    if x.correct_grouping:
        return x.grouping
    else:
        return tuple([x.roi_id])

def agg_amend(x):
    # first check if correct grouping; if it is, return grouping
    if x.correct_grouping:
        return x.grouping
    
    # else, get grouping from amended grouping col where roi_id occurs
    elif not x.correct_grouping:
        roi_id = x.roi_id
        grps = x[[i for i in x.index if 'amended' in i]].values
        grps = grps[~pd.isna(grps)]
        amended_grp = [grp for grp in grps if roi_id in grp]

        # for debugging
        if len(amended_grp) != 1:
            print(x)

        return amended_grp[0]

# apply aggregation function
paired_doubles['true_grouping'] = paired_doubles.apply(agg_amend_dbls, axis=1)
paired_triples['true_grouping'] = paired_triples.apply(agg_amend, axis=1)
paired_quads['true_grouping'] = paired_quads.apply(agg_amend, axis=1)

# let's take a look!
print(paired_quads.shape)
paired_quads.head()

(484, 13)


,index,Unnamed: 0,image_name,grouping,Correct grouping? (yes/no),amended grouping_1,amended grouping_2,amended grouping_3,ameded grouping_4,notes,correct_grouping,roi_id,true_grouping
0,0,0,KET-10-12_PFC_3.7_A,"(0-000-00012_PV, 0-005-00036_cFos, 0-FFF-00009...",yes,NaN,NaN,NaN,NaN,NaN,True,0-000-00012_PV,"(0-000-00012_PV, 0-005-00036_cFos, 0-FFF-00009..."
1,0,0,KET-10-12_PFC_3.7_A,"(0-000-00012_PV, 0-005-00036_cFos, 0-FFF-00009...",yes,NaN,NaN,NaN,NaN,NaN,True,0-005-00036_cFos,"(0-000-00012_PV, 0-005-00036_cFos, 0-FFF-00009..."
2,0,0,KET-10-12_PFC_3.7_A,"(0-000-00012_PV, 0-005-00036_cFos, 0-FFF-00009...",yes,NaN,NaN,NaN,NaN,NaN,True,0-FFF-00009_Npas4,"(0-000-00012_PV, 0-005-00036_cFos, 0-FFF-00009..."
3,0,0,KET-10-12_PFC_3.7_A,"(0-000-00012_PV, 0-005-00036_cFos, 0-FFF-00009...",yes,NaN,NaN,NaN,NaN,NaN,True,0-FFF-00001_WFA,"(0-000-00012_PV, 0-005-00036_cFos, 0-FFF-00009..."
4,1,1,KET-10-12_PFC_3.7_A,"(0-000-00000_PV, 0-005-00015_cFos, 0-FFF-00045...",no,"(0-000-00000_PV, 0-FFF-00045_Npas4, 0-FFF-0000...","(0-005-00015_cFos,)",NaN,NaN,NaN,False,0-000-00000_PV,"(0-000-00000_PV, 0-FFF-00045_Npas4, 0-FFF-0000..."


### selecting only the necessary cols, aggregating across amended quads/triples/doubles

In [4]:
paired_quads = paired_quads[['image_name', 'roi_id','true_grouping']]
paired_triples = paired_triples[['image_name', 'roi_id','true_grouping']]
paired_doubles = paired_doubles[['image_name', 'roi_id','true_grouping']]


df_amended = pd.concat([paired_quads, paired_triples, paired_doubles])

# get all the true_groupings for a given roi_id
df_amended = df_amended.groupby(by=['image_name', 'roi_id'])[['image_name', 'roi_id', 'true_grouping']]\
    .apply(lambda x: sorted(list(x.true_grouping.values), key=len, reverse=True))\
    .reset_index(name='groupings')

# aggregate true_groupings by taking the union of these sets (some true groupings overlap and imply a larger set)
df_amended['agg_grouping'] = df_amended.groupings.apply(lambda x: tuple(set.union(*[set(y) for y in x])))

df_amended

,image_name,roi_id,groupings,agg_grouping
0,KET-10-12_PFC_3.7_A,0-000-00000_PV,"[(0-000-00000_PV, 0-FFF-00045_Npas4, 0-FFF-000...","(0-FFF-00045_Npas4, 0-FFF-00004_WFA, 0-000-000..."
1,KET-10-12_PFC_3.7_A,0-000-00001_PV,"[(0-000-00001_PV, 0-FFF-00070_cFos, 0-FFF-0001...","(0-FFF-00012_Npas4, 0-000-00001_PV, 0-FFF-0007..."
2,KET-10-12_PFC_3.7_A,0-000-00012_PV,"[(0-000-00012_PV, 0-005-00036_cFos, 0-FFF-0000...","(0-005-00036_cFos, 0-FFF-00001_WFA, 0-000-0001..."
3,KET-10-12_PFC_3.7_A,0-000-00013_PV,"[(0-000-00013_PV, 0-005-00019_cFos, 0-FFF-0006...","(0-000-00013_PV, 0-005-00019_cFos, 0-FFF-00066..."
4,KET-10-12_PFC_3.7_A,0-005-00015_cFos,"[(0-005-00015_cFos,)]","(0-005-00015_cFos,)"
...,...,...,...,...
879,PE-13-9_PFC_3.3_C,0-FFF-00057_Npas4,"[(0-000-00004_PV, 0-FFF-00057_Npas4, 0-FFF-000...","(0-FFF-00057_Npas4, 0-FFF-00004_WFA, 0-000-000..."
880,PE-13-9_PFC_3.5_E,0-000-00001_PV,"[(0-000-00001_PV, 0-FFF-00034_cFos, 0-FFF-0000...","(0-FFF-00005_WFA, 0-FFF-00034_cFos, 0-000-0000..."
881,PE-13-9_PFC_3.5_E,0-FFF-00005_WFA,"[(0-000-00001_PV, 0-FFF-00034_cFos, 0-FFF-0000...","(0-FFF-00005_WFA, 0-FFF-00034_cFos, 0-000-0000..."
882,PE-13-9_PFC_3.5_E,0-FFF-00034_cFos,"[(0-000-00001_PV, 0-FFF-00034_cFos, 0-FFF-0000...","(0-FFF-00005_WFA, 0-FFF-00034_cFos, 0-000-0000..."


In [5]:
true_groupings = []
for img in df_amended.image_name.unique():
    amended_image = df_amended.query(f'image_name == "{img}"').copy(deep=True)

    # copy agg_grouping col to explode
    amended_image['exploded_roi_id'] = amended_image.agg_grouping

    # explode, groupby exploded roi_ids; gather all unique agg_groupings by exploded_roi_id
    amended_agg_groupings = amended_image.explode('exploded_roi_id')\
        .groupby('exploded_roi_id')['agg_grouping']\
        .unique().reset_index(name='overlapping_groupings')

    # create new true_grouping col by finding the intersection of each exploded_roi_id's arr of unique agg groupings
    amended_agg_groupings['true_grouping'] = amended_agg_groupings.overlapping_groupings.apply(lambda x: tuple(set.union(*[set(y) for y in x])))

    # rename exploded_roi_id as roi_id for merge
    amended_agg_groupings = amended_agg_groupings.rename(columns={'exploded_roi_id':'roi_id'})

    # merge with image copy
    true_group = amended_image.drop('exploded_roi_id', axis=1).merge(amended_agg_groupings, on='roi_id')

    # sort true_grouping tuple (cast as list, then sort, then cast back to tuple)
    true_group['true_grouping'] = true_group.true_grouping.apply(lambda x: tuple(sorted(list(x), key=lambda y: sort_order[y.split('_')[-1]])))

    # toss into list
    true_groupings.append(true_group)

df_amended_true = pd.concat(true_groupings)
df_amended_true    

,image_name,roi_id,groupings,agg_grouping,overlapping_groupings,true_grouping
0,KET-10-12_PFC_3.7_A,0-000-00000_PV,"[(0-000-00000_PV, 0-FFF-00045_Npas4, 0-FFF-000...","(0-FFF-00045_Npas4, 0-FFF-00004_WFA, 0-000-000...","[(0-FFF-00045_Npas4, 0-FFF-00004_WFA, 0-000-00...","(0-000-00000_PV, 0-FFF-00045_Npas4, 0-FFF-0000..."
1,KET-10-12_PFC_3.7_A,0-000-00001_PV,"[(0-000-00001_PV, 0-FFF-00070_cFos, 0-FFF-0001...","(0-FFF-00012_Npas4, 0-000-00001_PV, 0-FFF-0007...","[(0-FFF-00012_Npas4, 0-000-00001_PV, 0-FFF-000...","(0-000-00001_PV, 0-FFF-00070_cFos, 0-FFF-00012..."
2,KET-10-12_PFC_3.7_A,0-000-00012_PV,"[(0-000-00012_PV, 0-005-00036_cFos, 0-FFF-0000...","(0-005-00036_cFos, 0-FFF-00001_WFA, 0-000-0001...","[(0-005-00036_cFos, 0-FFF-00001_WFA, 0-000-000...","(0-000-00012_PV, 0-005-00036_cFos, 0-FFF-00009..."
3,KET-10-12_PFC_3.7_A,0-000-00013_PV,"[(0-000-00013_PV, 0-005-00019_cFos, 0-FFF-0006...","(0-000-00013_PV, 0-005-00019_cFos, 0-FFF-00066...","[(0-000-00013_PV, 0-005-00019_cFos, 0-FFF-0006...","(0-000-00013_PV, 0-005-00019_cFos, 0-FFF-00066..."
4,KET-10-12_PFC_3.7_A,0-005-00015_cFos,"[(0-005-00015_cFos,)]","(0-005-00015_cFos,)","[(0-005-00015_cFos,)]","(0-005-00015_cFos,)"
...,...,...,...,...,...,...
2,PE-13-9_PFC_3.3_C,0-FFF-00057_Npas4,"[(0-000-00004_PV, 0-FFF-00057_Npas4, 0-FFF-000...","(0-FFF-00057_Npas4, 0-FFF-00004_WFA, 0-000-000...","[(0-FFF-00057_Npas4, 0-FFF-00004_WFA, 0-000-00...","(0-000-00004_PV, 0-FFF-00057_Npas4, 0-FFF-0000..."
0,PE-13-9_PFC_3.5_E,0-000-00001_PV,"[(0-000-00001_PV, 0-FFF-00034_cFos, 0-FFF-0000...","(0-FFF-00005_WFA, 0-FFF-00034_cFos, 0-000-0000...","[(0-FFF-00005_WFA, 0-FFF-00034_cFos, 0-000-000...","(0-000-00001_PV, 0-FFF-00034_cFos, 0-FFF-00005..."
1,PE-13-9_PFC_3.5_E,0-FFF-00005_WFA,"[(0-000-00001_PV, 0-FFF-00034_cFos, 0-FFF-0000...","(0-FFF-00005_WFA, 0-FFF-00034_cFos, 0-000-0000...","[(0-FFF-00005_WFA, 0-FFF-00034_cFos, 0-000-000...","(0-000-00001_PV, 0-FFF-00034_cFos, 0-FFF-00005..."
2,PE-13-9_PFC_3.5_E,0-FFF-00034_cFos,"[(0-000-00001_PV, 0-FFF-00034_cFos, 0-FFF-0000...","(0-FFF-00005_WFA, 0-FFF-00034_cFos, 0-000-0000...","[(0-FFF-00005_WFA, 0-FFF-00034_cFos, 0-000-000...","(0-000-00001_PV, 0-FFF-00034_cFos, 0-FFF-00005..."


In [6]:
df_amended_true.to_csv('agg_amended.csv')

# Replacing old groupings with new amended groupings

## Building out a grouping col in our full df

In [7]:
sort_order = {'PV': 0, 'cFos': 1, 'Npas4': 2, 'WFA': 3}

images = []
for image in df_coloc.image_name.unique():

    # slice out image
    df_image = df_coloc.query(f'image_name == "{image}"').copy().reset_index().drop('index', axis=1)

    # build new grouping col of sorted tuples
    df_image['grouping'] = df_image.apply(\
        lambda x: tuple(sorted([\
            y for y in [x.roi_id, x['coloc_w/_PV'], x['coloc_w/_cFos'], x['coloc_w/_Npas4'], x['coloc_w/_WFA']] if not y == '-'],\
                key=lambda y: sort_order[y.split('_')[-1]])
    ), axis=1)

    # drop old dummy cols (these will be updated once i recieve updated coloc data from AG)
    df_image = df_image[[col for col in df_image.columns if not 'dummy' in col]]
    images.append(df_image)

images[0]

,Unnamed: 0,roi_id,coloc_w/_PV,coloc_w/_cFos,coloc_w/_Npas4,coloc_w/_WFA,CoM_x,CoM_y,background,mean_intensity,filename,rat_n,treatment,stain_type,image_name,grouping
0,0,0-000-00000_PV,-,-,0-FFF-00045_Npas4,0-FFF-00004_WFA,297.86,425.97,238.8715,536.8331,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,PV,KET-10-12_PFC_3.7_A,"(0-000-00000_PV, 0-FFF-00045_Npas4, 0-FFF-0000..."
1,1,0-000-00001_PV,-,0-FFF-00070_cFos,0-FFF-00012_Npas4,-,340.47,43.89,238.8715,314.9278,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,PV,KET-10-12_PFC_3.7_A,"(0-000-00001_PV, 0-FFF-00070_cFos, 0-FFF-00012..."
2,2,0-000-00002_PV,-,-,0-FFF-00044_Npas4,0-FFF-00005_WFA,154.85,476.15,238.8715,324.0556,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,PV,KET-10-12_PFC_3.7_A,"(0-000-00002_PV, 0-FFF-00044_Npas4, 0-FFF-0000..."
3,3,0-000-00003_PV,-,-,0-FFF-00082_Npas4,0-FFF-00003_WFA,310.10,308.22,238.8715,346.0313,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,PV,KET-10-12_PFC_3.7_A,"(0-000-00003_PV, 0-FFF-00082_Npas4, 0-FFF-0000..."
4,4,0-000-00004_PV,-,-,-,-,44.35,323.64,238.8715,429.6127,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,PV,KET-10-12_PFC_3.7_A,"(0-000-00004_PV,)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,204,0-FFF-00002_WFA,0-000-00008_PV,0-005-00017_cFos,0-FFF-00062_Npas4,-,172.04,315.17,115.7134,126.9843,KET-10-12_PFC_3.7_A_5.tif,KET-10-12,FR1_KET,WFA,KET-10-12_PFC_3.7_A,"(0-000-00008_PV, 0-005-00017_cFos, 0-FFF-00062..."
205,205,0-FFF-00003_WFA,0-000-00003_PV,-,0-FFF-00082_Npas4,-,311.54,305.09,115.7134,110.1349,KET-10-12_PFC_3.7_A_5.tif,KET-10-12,FR1_KET,WFA,KET-10-12_PFC_3.7_A,"(0-000-00003_PV, 0-FFF-00082_Npas4, 0-FFF-0000..."
206,206,0-FFF-00004_WFA,0-000-00000_PV,0-005-00015_cFos,0-FFF-00045_Npas4,-,297.09,427.72,115.7134,122.0008,KET-10-12_PFC_3.7_A_5.tif,KET-10-12,FR1_KET,WFA,KET-10-12_PFC_3.7_A,"(0-000-00000_PV, 0-005-00015_cFos, 0-FFF-00045..."
207,207,0-FFF-00005_WFA,0-000-00002_PV,-,0-FFF-00044_Npas4,-,157.10,478.13,115.7134,115.3184,KET-10-12_PFC_3.7_A_5.tif,KET-10-12,FR1_KET,WFA,KET-10-12_PFC_3.7_A,"(0-000-00002_PV, 0-FFF-00044_Npas4, 0-FFF-0000..."


## Replace with amended groupings

In [9]:
def replace_amended_groupings(x):
    iid = x.image_name
    rid = x.roi_id
    q = df_amended_true.query(f'image_name == "{iid}" and roi_id == "{rid}"')

    if not q.empty:
        # for debuggin (we expect len of exactly 1 here)
        if len(q) > 1:
            print(q)
        return q.true_grouping.values[0]
    
    elif q.empty:
        return x.grouping
    
df_coloc = pd.concat(images)
df_coloc['true_grouping'] = df_coloc.apply(replace_amended_groupings, axis=1)
df_coloc

,Unnamed: 0,roi_id,coloc_w/_PV,coloc_w/_cFos,coloc_w/_Npas4,coloc_w/_WFA,CoM_x,CoM_y,background,mean_intensity,filename,rat_n,treatment,stain_type,image_name,grouping,true_grouping
0,0,0-000-00000_PV,-,-,0-FFF-00045_Npas4,0-FFF-00004_WFA,297.86,425.97,238.8715,536.8331,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,PV,KET-10-12_PFC_3.7_A,"(0-000-00000_PV, 0-FFF-00045_Npas4, 0-FFF-0000...","(0-000-00000_PV, 0-FFF-00045_Npas4, 0-FFF-0000..."
1,1,0-000-00001_PV,-,0-FFF-00070_cFos,0-FFF-00012_Npas4,-,340.47,43.89,238.8715,314.9278,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,PV,KET-10-12_PFC_3.7_A,"(0-000-00001_PV, 0-FFF-00070_cFos, 0-FFF-00012...","(0-000-00001_PV, 0-FFF-00070_cFos, 0-FFF-00012..."
2,2,0-000-00002_PV,-,-,0-FFF-00044_Npas4,0-FFF-00005_WFA,154.85,476.15,238.8715,324.0556,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,PV,KET-10-12_PFC_3.7_A,"(0-000-00002_PV, 0-FFF-00044_Npas4, 0-FFF-0000...","(0-000-00002_PV, 0-FFF-00044_Npas4, 0-FFF-0000..."
3,3,0-000-00003_PV,-,-,0-FFF-00082_Npas4,0-FFF-00003_WFA,310.10,308.22,238.8715,346.0313,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,PV,KET-10-12_PFC_3.7_A,"(0-000-00003_PV, 0-FFF-00082_Npas4, 0-FFF-0000...","(0-000-00003_PV, 0-FFF-00082_Npas4, 0-FFF-0000..."
4,4,0-000-00004_PV,-,-,-,-,44.35,323.64,238.8715,429.6127,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,PV,KET-10-12_PFC_3.7_A,"(0-000-00004_PV,)","(0-000-00004_PV,)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,22777,0-FFF-00008_WFA,-,-,-,-,385.12,235.30,109.6097,119.2211,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,WFA,PE-13-9_PFC_4.0_B,"(0-FFF-00008_WFA,)","(0-FFF-00008_WFA,)"
119,22778,0-FFF-00009_WFA,0-000-00002_PV,-,-,-,409.79,230.66,109.6097,128.8761,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,WFA,PE-13-9_PFC_4.0_B,"(0-000-00002_PV, 0-FFF-00009_WFA)","(0-000-00002_PV, 0-FFF-00009_WFA)"
120,22779,0-FFF-00010_WFA,-,-,-,-,300.24,100.38,109.6097,123.8280,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,WFA,PE-13-9_PFC_4.0_B,"(0-FFF-00010_WFA,)","(0-FFF-00010_WFA,)"
121,22780,0-FFF-00011_WFA,0-000-00005_PV,-,-,-,414.17,44.01,109.6097,120.4709,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,WFA,PE-13-9_PFC_4.0_B,"(0-000-00005_PV, 0-FFF-00011_WFA)","(0-000-00005_PV, 0-FFF-00011_WFA)"


## building new dummy cols

In [10]:
def get_dummies(x):
    groupings = [rid.split('_')[-1] for rid in x]

    dummy_PV = False
    dummy_cFos = False
    dummy_Npas4 = False
    dummy_WFA = False

    if 'PV' in groupings:
        dummy_PV = True
    if 'cFos' in groupings:
        dummy_cFos = True
    if 'Npas4' in groupings:
        dummy_Npas4 = True
    if 'WFA' in groupings:
        dummy_WFA = True

    return dummy_PV, dummy_cFos, dummy_Npas4, dummy_WFA

df_coloc['dummy'] = df_coloc.true_grouping.apply(get_dummies)
df_coloc['dummy_PV'], df_coloc['dummy_cFos'], df_coloc['dummy_Npas4'], df_coloc['dummy_WFA'] = zip(*df_coloc['dummy'])

df_coloc

,Unnamed: 0,roi_id,coloc_w/_PV,coloc_w/_cFos,coloc_w/_Npas4,coloc_w/_WFA,CoM_x,CoM_y,background,mean_intensity,...,treatment,stain_type,image_name,grouping,true_grouping,dummy,dummy_PV,dummy_cFos,dummy_Npas4,dummy_WFA
0,0,0-000-00000_PV,-,-,0-FFF-00045_Npas4,0-FFF-00004_WFA,297.86,425.97,238.8715,536.8331,...,FR1_KET,PV,KET-10-12_PFC_3.7_A,"(0-000-00000_PV, 0-FFF-00045_Npas4, 0-FFF-0000...","(0-000-00000_PV, 0-FFF-00045_Npas4, 0-FFF-0000...","(True, False, True, True)",True,False,True,True
1,1,0-000-00001_PV,-,0-FFF-00070_cFos,0-FFF-00012_Npas4,-,340.47,43.89,238.8715,314.9278,...,FR1_KET,PV,KET-10-12_PFC_3.7_A,"(0-000-00001_PV, 0-FFF-00070_cFos, 0-FFF-00012...","(0-000-00001_PV, 0-FFF-00070_cFos, 0-FFF-00012...","(True, True, True, False)",True,True,True,False
2,2,0-000-00002_PV,-,-,0-FFF-00044_Npas4,0-FFF-00005_WFA,154.85,476.15,238.8715,324.0556,...,FR1_KET,PV,KET-10-12_PFC_3.7_A,"(0-000-00002_PV, 0-FFF-00044_Npas4, 0-FFF-0000...","(0-000-00002_PV, 0-FFF-00044_Npas4, 0-FFF-0000...","(True, False, True, True)",True,False,True,True
3,3,0-000-00003_PV,-,-,0-FFF-00082_Npas4,0-FFF-00003_WFA,310.10,308.22,238.8715,346.0313,...,FR1_KET,PV,KET-10-12_PFC_3.7_A,"(0-000-00003_PV, 0-FFF-00082_Npas4, 0-FFF-0000...","(0-000-00003_PV, 0-FFF-00082_Npas4, 0-FFF-0000...","(True, False, True, True)",True,False,True,True
4,4,0-000-00004_PV,-,-,-,-,44.35,323.64,238.8715,429.6127,...,FR1_KET,PV,KET-10-12_PFC_3.7_A,"(0-000-00004_PV,)","(0-000-00004_PV,)","(True, False, False, False)",True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,22777,0-FFF-00008_WFA,-,-,-,-,385.12,235.30,109.6097,119.2211,...,VR5_SAL,WFA,PE-13-9_PFC_4.0_B,"(0-FFF-00008_WFA,)","(0-FFF-00008_WFA,)","(False, False, False, True)",False,False,False,True
119,22778,0-FFF-00009_WFA,0-000-00002_PV,-,-,-,409.79,230.66,109.6097,128.8761,...,VR5_SAL,WFA,PE-13-9_PFC_4.0_B,"(0-000-00002_PV, 0-FFF-00009_WFA)","(0-000-00002_PV, 0-FFF-00009_WFA)","(True, False, False, True)",True,False,False,True
120,22779,0-FFF-00010_WFA,-,-,-,-,300.24,100.38,109.6097,123.8280,...,VR5_SAL,WFA,PE-13-9_PFC_4.0_B,"(0-FFF-00010_WFA,)","(0-FFF-00010_WFA,)","(False, False, False, True)",False,False,False,True
121,22780,0-FFF-00011_WFA,0-000-00005_PV,-,-,-,414.17,44.01,109.6097,120.4709,...,VR5_SAL,WFA,PE-13-9_PFC_4.0_B,"(0-000-00005_PV, 0-FFF-00011_WFA)","(0-000-00005_PV, 0-FFF-00011_WFA)","(True, False, False, True)",True,False,False,True


Some interesting timeit tests

In [11]:
%timeit df_coloc['dummy_PV'], df_coloc['dummy_cFos'], df_coloc['dummy_Npas4'], df_coloc['dummy_WFA'] = zip(*df_coloc['dummy'])
%timeit df_coloc.assign(**dict(zip(['dummy_PV', 'dummy_cFos', 'dummy_Npas4', 'dummy_WFA'], zip(*df_coloc.dummy))))


8.16 ms ± 231 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.3 ms ± 192 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Moving on to analyses
## just doing some more preliminary cleaning

In [12]:
df_coloc = df_coloc[['filename', 'image_name', 'roi_id', 'true_grouping', \
                     'dummy_PV', 'dummy_cFos', 'dummy_Npas4', 'dummy_WFA',\
                     'CoM_x', 'CoM_y', 'background', 'mean_intensity', \
                     'stain_type', 'filename', 'rat_n', 'treatment']]\
                     .copy(deep=True).reset_index()

df_coloc['group_name'] = df_coloc.rat_n.apply(lambda x: '-'.join(x.split('-')[:2]))

# drop everything from group PE-11
df_coloc = df_coloc.query('group_name != "PE-11"')

print(df_coloc.shape)
df_coloc

(18632, 18)


,index,filename,image_name,roi_id,true_grouping,dummy_PV,dummy_cFos,dummy_Npas4,dummy_WFA,CoM_x,CoM_y,background,mean_intensity,stain_type,filename,rat_n,treatment,group_name
0,0,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00000_PV,"(0-000-00000_PV, 0-FFF-00045_Npas4, 0-FFF-0000...",True,False,True,True,297.86,425.97,238.8715,536.8331,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10
1,1,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00001_PV,"(0-000-00001_PV, 0-FFF-00070_cFos, 0-FFF-00012...",True,True,True,False,340.47,43.89,238.8715,314.9278,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10
2,2,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00002_PV,"(0-000-00002_PV, 0-FFF-00044_Npas4, 0-FFF-0000...",True,False,True,True,154.85,476.15,238.8715,324.0556,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10
3,3,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00003_PV,"(0-000-00003_PV, 0-FFF-00082_Npas4, 0-FFF-0000...",True,False,True,True,310.10,308.22,238.8715,346.0313,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10
4,4,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00004_PV,"(0-000-00004_PV,)",True,False,False,False,44.35,323.64,238.8715,429.6127,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22793,118,PE-13-9_PFC_4.0_B_5.tif,PE-13-9_PFC_4.0_B,0-FFF-00008_WFA,"(0-FFF-00008_WFA,)",False,False,False,True,385.12,235.30,109.6097,119.2211,WFA,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,PE-13
22794,119,PE-13-9_PFC_4.0_B_5.tif,PE-13-9_PFC_4.0_B,0-FFF-00009_WFA,"(0-000-00002_PV, 0-FFF-00009_WFA)",True,False,False,True,409.79,230.66,109.6097,128.8761,WFA,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,PE-13
22795,120,PE-13-9_PFC_4.0_B_5.tif,PE-13-9_PFC_4.0_B,0-FFF-00010_WFA,"(0-FFF-00010_WFA,)",False,False,False,True,300.24,100.38,109.6097,123.8280,WFA,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,PE-13
22796,121,PE-13-9_PFC_4.0_B_5.tif,PE-13-9_PFC_4.0_B,0-FFF-00011_WFA,"(0-000-00005_PV, 0-FFF-00011_WFA)",True,False,False,True,414.17,44.01,109.6097,120.4709,WFA,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,PE-13


## SNR Threshold

For each stain type we determined the following SNR thresholds:
- PV : 0.8
- cFos : 0
- Npas4 : 0.8
- WFA : 0.85

We did this already, but given that a whole group of animals was removed, we should repeat this step. Given that the data we loaded has already been snr thresholded, we expect the same number of cells before and after filtering.

In [13]:
def get_snr(df):
    '''
    '''
    df['snr'] = df['mean_intensity'].astype('f') / df['background'].astype('f')

    return df

def filter_snr(df, snr_cutoff):
    '''
    '''
    df_threshold = pd.concat([df.query(f'stain_type == "{stain}"').query(f'snr > {snr_cutoff[stain]}') for stain in df.stain_type.unique()])

    return df_threshold

# build dict of snr thresholds
snr_threshold = {
    'PV': 0.8,
    'cFos': 0,
    'Npas4': 0.8,
    'WFA': 0.85
}

df_coloc = filter_snr(get_snr(df_coloc), snr_threshold)

print(df_coloc.shape)
df_coloc

(18632, 19)


,index,filename,image_name,roi_id,true_grouping,dummy_PV,dummy_cFos,dummy_Npas4,dummy_WFA,CoM_x,CoM_y,background,mean_intensity,stain_type,filename,rat_n,treatment,group_name,snr
0,0,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00000_PV,"(0-000-00000_PV, 0-FFF-00045_Npas4, 0-FFF-0000...",True,False,True,True,297.86,425.97,238.8715,536.8331,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10,2.247372
1,1,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00001_PV,"(0-000-00001_PV, 0-FFF-00070_cFos, 0-FFF-00012...",True,True,True,False,340.47,43.89,238.8715,314.9278,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10,1.318398
2,2,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00002_PV,"(0-000-00002_PV, 0-FFF-00044_Npas4, 0-FFF-0000...",True,False,True,True,154.85,476.15,238.8715,324.0556,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10,1.356611
3,3,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00003_PV,"(0-000-00003_PV, 0-FFF-00082_Npas4, 0-FFF-0000...",True,False,True,True,310.10,308.22,238.8715,346.0313,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10,1.448609
4,4,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00004_PV,"(0-000-00004_PV,)",True,False,False,False,44.35,323.64,238.8715,429.6127,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10,1.798510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22793,118,PE-13-9_PFC_4.0_B_5.tif,PE-13-9_PFC_4.0_B,0-FFF-00008_WFA,"(0-FFF-00008_WFA,)",False,False,False,True,385.12,235.30,109.6097,119.2211,WFA,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,PE-13,1.087687
22794,119,PE-13-9_PFC_4.0_B_5.tif,PE-13-9_PFC_4.0_B,0-FFF-00009_WFA,"(0-000-00002_PV, 0-FFF-00009_WFA)",True,False,False,True,409.79,230.66,109.6097,128.8761,WFA,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,PE-13,1.175773
22795,120,PE-13-9_PFC_4.0_B_5.tif,PE-13-9_PFC_4.0_B,0-FFF-00010_WFA,"(0-FFF-00010_WFA,)",False,False,False,True,300.24,100.38,109.6097,123.8280,WFA,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,PE-13,1.129718
22796,121,PE-13-9_PFC_4.0_B_5.tif,PE-13-9_PFC_4.0_B,0-FFF-00011_WFA,"(0-000-00005_PV, 0-FFF-00011_WFA)",True,False,False,True,414.17,44.01,109.6097,120.4709,WFA,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,PE-13,1.099090


## Drop nans, check for duplicates

In [14]:
# which cols have nans, how many?
print('Nan per col:')
print(df_coloc.isna().sum())

# how many duplicated rows do we have?
print('\nTotal n of duplicated rows:')

# it's best to count duplicates on the numeric data to avoid false negatives due to 
# errors in spelling or systematic labeling
print(df_coloc.duplicated(subset=['filename', 'image_name', 'CoM_x', 'CoM_y',
       'background', 'mean_intensity', 'stain_type', 'filename', 'rat_n',
       'treatment', 'group_name', 'snr']).sum())

Nan per col:
index             0
filename          0
image_name        0
roi_id            0
true_grouping     0
dummy_PV          0
dummy_cFos        0
dummy_Npas4       0
dummy_WFA         0
CoM_x             0
CoM_y             0
background        0
mean_intensity    0
stain_type        0
filename          0
rat_n             0
treatment         0
group_name        0
snr               0
dtype: int64

Total n of duplicated rows:
0


## Adjusting mean-background

Just to be sure that adjusting the mean-background with out PE-11, I inspected the original csv called "VR5_COLOC_FULL_DB.csv" and for each of the stain types, I have the minimum observed mean_intensity occurring in the following images:
- PV: 0-FFF-00006_PV from KET-10-1_PFC_3.7_B_2.tif
- cFos: 0-FFF-00008_cFos from PE-13-11_PFC_3.7_A_3.tif
- Npas4: 0-FFF-00001_Npas4 from KET-9-5_PFC_3.5_B_4.tif
- WFA: 0-FFF-00001_WFA from PE-13-11_PFC_3.7_A_5.tif

Since none of the dimmest cells came from in PE-11, the adjustment calculation is exactly the same as before. 

In [15]:
def adjust_mmbg(df):
    '''
    '''
    # remove unnecesary index col
    if 'index' in df.columns:
        df = df.drop('index', axis=1)

    # add minimum observed intensity 
    adjusted_mmbg = []
    for stain in df.stain_type.unique():
        # separate by stain
        df_stain = df.query(f'stain_type == "{stain}"').reset_index()

        # compute new mean-background col
        df_stain['mean-background'] = df_stain.loc[: ,'mean_intensity'].astype('f') - df_stain.loc[:, 'background'].astype('f')

        # get min mean-background (some negative number)
        min_mmbg = df_stain['mean-background'].min()

        # add the absolute value of min to all other cells of the same stain
        df_stain['adjusted_mean-background'] = df_stain['mean-background'] + abs(min_mmbg)

        # toss back into list
        adjusted_mmbg.append(df_stain)

    df_adjusted = pd.concat(adjusted_mmbg)

    return df_adjusted

df_adjusted = adjust_mmbg(df_coloc)
print(df_adjusted.shape)
df_adjusted.head()

(18632, 21)


,index,filename,image_name,roi_id,true_grouping,dummy_PV,dummy_cFos,dummy_Npas4,dummy_WFA,CoM_x,...,background,mean_intensity,stain_type,filename,rat_n,treatment,group_name,snr,mean-background,adjusted_mean-background
0,0,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00000_PV,"(0-000-00000_PV, 0-FFF-00045_Npas4, 0-FFF-0000...",True,False,True,True,297.86,...,238.8715,536.8331,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10,2.247372,297.961609,382.724609
1,1,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00001_PV,"(0-000-00001_PV, 0-FFF-00070_cFos, 0-FFF-00012...",True,True,True,False,340.47,...,238.8715,314.9278,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10,1.318398,76.056290,160.819290
2,2,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00002_PV,"(0-000-00002_PV, 0-FFF-00044_Npas4, 0-FFF-0000...",True,False,True,True,154.85,...,238.8715,324.0556,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10,1.356611,85.184097,169.947098
3,3,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00003_PV,"(0-000-00003_PV, 0-FFF-00082_Npas4, 0-FFF-0000...",True,False,True,True,310.10,...,238.8715,346.0313,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10,1.448609,107.159805,191.922806
4,4,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00004_PV,"(0-000-00004_PV,)",True,False,False,False,44.35,...,238.8715,429.6127,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10,1.798510,190.741196,275.504211


In [16]:
df_adjusted = df_adjusted.drop('index', axis=1)
df_adjusted = df_adjusted.reset_index()
df_adjusted

,index,filename,image_name,roi_id,true_grouping,dummy_PV,dummy_cFos,dummy_Npas4,dummy_WFA,CoM_x,...,background,mean_intensity,stain_type,filename,rat_n,treatment,group_name,snr,mean-background,adjusted_mean-background
0,0,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00000_PV,"(0-000-00000_PV, 0-FFF-00045_Npas4, 0-FFF-0000...",True,False,True,True,297.86,...,238.8715,536.8331,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10,2.247372,297.961609,382.724609
1,1,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00001_PV,"(0-000-00001_PV, 0-FFF-00070_cFos, 0-FFF-00012...",True,True,True,False,340.47,...,238.8715,314.9278,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10,1.318398,76.056290,160.819290
2,2,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00002_PV,"(0-000-00002_PV, 0-FFF-00044_Npas4, 0-FFF-0000...",True,False,True,True,154.85,...,238.8715,324.0556,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10,1.356611,85.184097,169.947098
3,3,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00003_PV,"(0-000-00003_PV, 0-FFF-00082_Npas4, 0-FFF-0000...",True,False,True,True,310.10,...,238.8715,346.0313,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10,1.448609,107.159805,191.922806
4,4,KET-10-12_PFC_3.7_A_2.tif,KET-10-12_PFC_3.7_A,0-000-00004_PV,"(0-000-00004_PV,)",True,False,False,False,44.35,...,238.8715,429.6127,PV,KET-10-12_PFC_3.7_A_2.tif,KET-10-12,FR1_KET,KET-10,1.798510,190.741196,275.504211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18627,898,PE-13-9_PFC_4.0_B_5.tif,PE-13-9_PFC_4.0_B,0-FFF-00008_WFA,"(0-FFF-00008_WFA,)",False,False,False,True,385.12,...,109.6097,119.2211,WFA,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,PE-13,1.087687,9.611397,26.986496
18628,899,PE-13-9_PFC_4.0_B_5.tif,PE-13-9_PFC_4.0_B,0-FFF-00009_WFA,"(0-000-00002_PV, 0-FFF-00009_WFA)",True,False,False,True,409.79,...,109.6097,128.8761,WFA,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,PE-13,1.175773,19.266396,36.641495
18629,900,PE-13-9_PFC_4.0_B_5.tif,PE-13-9_PFC_4.0_B,0-FFF-00010_WFA,"(0-FFF-00010_WFA,)",False,False,False,True,300.24,...,109.6097,123.8280,WFA,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,PE-13,1.129718,14.218300,31.593399
18630,901,PE-13-9_PFC_4.0_B_5.tif,PE-13-9_PFC_4.0_B,0-FFF-00011_WFA,"(0-000-00005_PV, 0-FFF-00011_WFA)",True,False,False,True,414.17,...,109.6097,120.4709,WFA,PE-13-9_PFC_4.0_B_5.tif,PE-13-9,VR5_SAL,PE-13,1.099090,10.861198,28.236298


## Filling in some dummy_WFA entries

In [17]:
WFA_bool_sum = np.array(df_adjusted.stain_type == 'WFA') + np.array(df_adjusted.dummy_WFA)
df_adjusted['dummy_WFA'] = WFA_bool_sum

#df_adjusted = df_adjusted.reset_index()

# filling in cells I should have earlier
lonely_WFA_i = df_adjusted.query('roi_id == "-"').index
for i in lonely_WFA_i:
    df_adjusted.at[i, 'roi_id'] = f'{i}_amended_WFA'
    df_adjusted.at[i, 'true_grouping'] = tuple([f'{i}_amended_WFA'])
    df_adjusted.at[i, 'dummy_WFA'] = True


df_adjusted[df_adjusted.roi_id.str.contains('amended')]


,index,filename,image_name,roi_id,true_grouping,dummy_PV,dummy_cFos,dummy_Npas4,dummy_WFA,CoM_x,...,background,mean_intensity,stain_type,filename,rat_n,treatment,group_name,snr,mean-background,adjusted_mean-background
18341,612,PE-13-11_PFC_3.3_C_5.tif,PE-13-11_PFC_3.3_C,18341_amended_WFA,"(18341_amended_WFA,)",False,False,False,True,177.27,...,93.9887,88.2249,WFA,PE-13-11_PFC_3.3_C_5.tif,PE-13-11,VR5_SAL,PE-13,0.938676,-5.763802,11.611298
18356,627,PE-13-11_PFC_3.5_D_5.tif,PE-13-11_PFC_3.5_D,18356_amended_WFA,"(18356_amended_WFA,)",False,False,False,True,164.01,...,111.9724,146.2031,WFA,PE-13-11_PFC_3.5_D_5.tif,PE-13-11,VR5_SAL,PE-13,1.305707,34.230698,51.605797
18357,628,PE-13-11_PFC_3.5_D_5.tif,PE-13-11_PFC_3.5_D,18357_amended_WFA,"(18357_amended_WFA,)",False,False,False,True,197.94,...,111.9724,123.3614,WFA,PE-13-11_PFC_3.5_D_5.tif,PE-13-11,VR5_SAL,PE-13,1.101713,11.389000,28.764099
18358,629,PE-13-11_PFC_3.5_D_5.tif,PE-13-11_PFC_3.5_D,18358_amended_WFA,"(18358_amended_WFA,)",False,False,False,True,229.32,...,111.9724,108.3670,WFA,PE-13-11_PFC_3.5_D_5.tif,PE-13-11,VR5_SAL,PE-13,0.967801,-3.605400,13.769699
18359,630,PE-13-11_PFC_3.5_D_5.tif,PE-13-11_PFC_3.5_D,18359_amended_WFA,"(18359_amended_WFA,)",False,False,False,True,261.34,...,111.9724,106.3260,WFA,PE-13-11_PFC_3.5_D_5.tif,PE-13-11,VR5_SAL,PE-13,0.949573,-5.646400,11.728699
18387,658,PE-13-11_PFC_3.7_B_5.tif,PE-13-11_PFC_3.7_B,18387_amended_WFA,"(18387_amended_WFA,)",False,False,False,True,259.31,...,106.0961,106.9537,WFA,PE-13-11_PFC_3.7_B_5.tif,PE-13-11,VR5_SAL,PE-13,1.008083,0.857597,18.232697
18388,659,PE-13-11_PFC_3.7_B_5.tif,PE-13-11_PFC_3.7_B,18388_amended_WFA,"(18388_amended_WFA,)",False,False,False,True,190.44,...,106.0961,99.3272,WFA,PE-13-11_PFC_3.7_B_5.tif,PE-13-11,VR5_SAL,PE-13,0.936200,-6.768898,10.606201
18389,660,PE-13-11_PFC_3.7_B_5.tif,PE-13-11_PFC_3.7_B,18389_amended_WFA,"(18389_amended_WFA,)",False,False,False,True,315.86,...,106.0961,113.1720,WFA,PE-13-11_PFC_3.7_B_5.tif,PE-13-11,VR5_SAL,PE-13,1.066693,7.075897,24.450996
18390,661,PE-13-11_PFC_3.7_B_5.tif,PE-13-11_PFC_3.7_B,18390_amended_WFA,"(18390_amended_WFA,)",False,False,False,True,264.03,...,106.0961,97.4909,WFA,PE-13-11_PFC_3.7_B_5.tif,PE-13-11,VR5_SAL,PE-13,0.918892,-8.605202,8.769897
18474,745,PE-13-2_PFC_3.5_B_5.tif,PE-13-2_PFC_3.5_B,18474_amended_WFA,"(18474_amended_WFA,)",False,False,False,True,343.08,...,101.4327,102.1607,WFA,PE-13-2_PFC_3.5_B_5.tif,PE-13-2,VR5_KET,PE-13,1.007177,0.727997,18.103096


In [18]:
import itertools
stains = np.array(['PV', 'cFos', 'Npas4', 'WFA'])

coloc_stain_types = ['PV', 'cFos', 'Npas4', 'WFA',\
                     'quad_PV', 'quad_cFos', 'quad_Npas4', 'quad_WFA']
coloc_stain_types = [tuple([s]) for s in coloc_stain_types]
for stain in stains:
    self = stain
    nonself = np.delete(stains, np.argwhere(stains == self))

    # double combinations
    dbl_comb = list(itertools.product([self],itertools.combinations(nonself, r=1)))

    # triple combinations
    trp_comb = list(itertools.product([self],itertools.combinations(nonself, r=2)))

    coloc_stain_types += dbl_comb
    coloc_stain_types += trp_comb

coloc_stain_types

[('PV',),
 ('cFos',),
 ('Npas4',),
 ('WFA',),
 ('quad_PV',),
 ('quad_cFos',),
 ('quad_Npas4',),
 ('quad_WFA',),
 ('PV', ('cFos',)),
 ('PV', ('Npas4',)),
 ('PV', ('WFA',)),
 ('PV', ('cFos', 'Npas4')),
 ('PV', ('cFos', 'WFA')),
 ('PV', ('Npas4', 'WFA')),
 ('cFos', ('PV',)),
 ('cFos', ('Npas4',)),
 ('cFos', ('WFA',)),
 ('cFos', ('PV', 'Npas4')),
 ('cFos', ('PV', 'WFA')),
 ('cFos', ('Npas4', 'WFA')),
 ('Npas4', ('PV',)),
 ('Npas4', ('cFos',)),
 ('Npas4', ('WFA',)),
 ('Npas4', ('PV', 'cFos')),
 ('Npas4', ('PV', 'WFA')),
 ('Npas4', ('cFos', 'WFA')),
 ('WFA', ('PV',)),
 ('WFA', ('cFos',)),
 ('WFA', ('Npas4',)),
 ('WFA', ('PV', 'cFos')),
 ('WFA', ('PV', 'Npas4')),
 ('WFA', ('cFos', 'Npas4'))]

In [20]:
df_adjusted.to_csv('KET-VR5_FULL_SET.csv')

## Please see accompanying notebook called "Combing through cell counts"